In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Any results you write to the current directory are saved as output.

In [38]:
class Node:

    def __init__(self, x, y, idxs, min_leaf=5):
        self.x = x 
        self.y = y
        self.idxs = idxs 
        self.min_leaf = min_leaf
        self.row_count = len(idxs)
        self.col_count = x.shape[1]
        self.val = np.mean(y[idxs])
        self.score = float('inf')
        self.find_varsplit()
        
    def find_varsplit(self):
        for c in range(self.col_count): self.find_better_split(c)
        if self.is_leaf: return
        x = self.split_col
        lhs = np.nonzero(x <= self.split)[0]
        rhs = np.nonzero(x > self.split)[0]
        self.lhs = Node(self.x, self.y, self.idxs[lhs], self.min_leaf)
        self.rhs = Node(self.x, self.y, self.idxs[rhs], self.min_leaf)
        
    def find_better_split(self, var_idx):
      
        x = self.x[self.idxs, var_idx]

        for r in range(self.row_count):
            lhs = x <= x[r]
            rhs = x > x[r]
            if rhs.sum() < self.min_leaf or lhs.sum() < self.min_leaf: continue

            curr_score = self.find_score(lhs, rhs)
            if curr_score < self.score: 
                self.var_idx = var_idx
                self.score = curr_score
                self.split = x[r]
                
    def find_score(self, lhs, rhs):
        y = self.y[self.idxs]
        lhs_std = y[lhs].std()
        rhs_std = y[rhs].std()
        return lhs_std * lhs.sum() + rhs_std * rhs.sum()
                
    @property
    def split_col(self): return self.x[self.idxs,self.var_idx]
                
    @property
    def is_leaf(self): return self.score == float('inf')                

    def predict(self, x):
        return np.array([self.predict_row(xi) for xi in x])

    def predict_row(self, xi):
        if self.is_leaf: return self.val
        node = self.lhs if xi[self.var_idx] <= self.split else self.rhs
        return node.predict_row(xi)


In [51]:
class DecisionTreeRegressor:
  
  def fit(self, X, y, min_leaf = 5):
    self.dtree = Node(X, y, np.array(np.arange(len(y))), min_leaf)
    return self
  
  def predict(self, X):
    return self.dtree.predict(X)

In [52]:
trainD = np.array(pd.read_csv("/home/somani/PycharmProjects/MLRG1/Assignment_4/train.csv", header=0, delimiter = ','))
print(trainD.shape)

(27000, 15)


In [53]:
print(trainD[0][1])

 State-gov


In [54]:
def encodeCol(ar,col):
    p = ar[:,col]
    i = 1
    dic = {}
    
    for val in range(len(p)):
        s = p[val]
        if s in dic:
            p[val]=dic[s]
        else:
            dic[s]=i
            p[val]=i
            i=i+1
    
    return p

In [55]:
trainX = trainD[:,0:14]
trainY = trainD[:,14]


In [56]:
trainX[:,1] = encodeCol(trainX,1)
trainX[:,3] = encodeCol(trainX,3)
trainX[:,5] = encodeCol(trainX,5)
trainX[:,6] = encodeCol(trainX,6)
trainX[:,7] = encodeCol(trainX,7)
trainX[:,8] = encodeCol(trainX,8)
trainX[:,9] = encodeCol(trainX,9)
trainX[:,13] = encodeCol(trainX,13)


In [45]:
regressor = DecisionTreeRegressor().fit(trainX, trainY)
preds = regressor.predict(trainX)

In [46]:
print(preds)

[0.  0.  0.  ... 0.  0.5 0. ]


In [48]:
delt = trainY-preds
delt = np.abs(delt)

rou = np.round(preds)



In [49]:
delr = np.abs(trainY-rou)
print(np.sum(delr))

2183.0


In [50]:
print((27000-2183)*100/27000)

91.91481481481482
